In [1]:
import pandas as pd
import sql_functions as sql_int
import Levenshtein

## Get Names from Tournament


take the playerlist and reduce duplikates in it

In [2]:
player_import = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/name_list_GSC1.csv',
                         dtype={'liquipedia_name' : str,'coutry_flag' : str} )
df_player_import = player_import.copy()

In [4]:
df_player_import = df_player_import.rename(columns={'liquipedia_name' : 'import_name'})
df_player_import['import_name_lower'] = df_player_import['import_name'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df_player_import = df_player_import.drop_duplicates()

Same liquipedia name but different country correct the entrys

In [7]:
df_player_import[df_player_import.duplicated(keep = False, subset=['import_name']) == True].sort_values(by='import_name')

,import_name,import_name_lower


#### delete double entrys

In [5]:
df_player_import.drop([0],inplace=True)


## Compare Names with *alternative_names*

In [17]:
schema = 'tft_tables'
sql_query = f'''select player_name, liquipedia_name, names_id, a.player_id 
                FROM alternative_names a 
                LEFT JOIN players p ON p.player_id = a.player_id ;
                '''

alternative_names = pd.DataFrame(sql_int.get_data_mysql(sql_query),
                                    columns=['player_name','liquipedia_name','names_id','player_id'])
df_alternative_names = alternative_names.copy()

df_alternative_names[['player_name','liquipedia_name']] = df_alternative_names[['player_name','liquipedia_name']].apply(lambda x: x.str.lower())


replace the imported names with liquipedia_names from the alternative list.   
If there was someone with his alternative name in, they have now liquipedia_name

In [9]:
df_merge_import_alternative = (pd.merge(df_player_import['import_name_lower'], df_alternative_names[['player_name','liquipedia_name']], how="left", left_on='import_name_lower',right_on='player_name')
                               .rename(columns={'import_name_lower' : 'import_name'}))

df_merge_import_alternative.loc[df_merge_import_alternative['player_name'].notna(), 'import_name'] = df_merge_import_alternative['liquipedia_name']
df_merge_import_alternative = df_merge_import_alternative.drop(columns=['player_name','liquipedia_name']).drop_duplicates()
df_merge_import_alternative

,import_name
0,lelouch
1,wet jungler
2,lastkardax
3,l3s coco
4,tsarou
...,...
123,hyunter
124,swellertiger
125,banefirexd
126,omino blue


2nd check cause 2nd merge

In [10]:
df_merge_import_alternative[df_merge_import_alternative.duplicated(keep= False, subset=['import_name']) == True].sort_values(by='import_name')

,import_name


## Search Names in *players*

In [11]:
sql_query = f'''select liquipedia_name, player_id from {schema}.players;'''


liquipedia_names = pd.DataFrame(sql_int.get_data_mysql(sql_query),
                                columns=['liquipedia_name','player_id'])

df_liquipedia_names = liquipedia_names.copy()
df_liquipedia_names['liquipedia_name'] = df_liquipedia_names['liquipedia_name'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df_merge_import_alternatives_liquipedia =pd.merge(
                                            df_merge_import_alternative,df_liquipedia_names['liquipedia_name'], how='left',
                                            left_on= 'import_name', right_on='liquipedia_name')

df_new_players are all players we put in over our names_list, which arn't in *players* 

In [12]:
df_new_players = df_merge_import_alternatives_liquipedia[df_merge_import_alternatives_liquipedia['liquipedia_name'] 
                                                         .isna()].drop(columns=['liquipedia_name'])

## Push new Names to *alternative_names*

In [13]:
list_new_players = df_new_players['import_name'].to_list()
list_liquipedia_name = df_liquipedia_names['liquipedia_name'].to_list()

In [14]:
list_new_players

[]

 Check similarity 

In [15]:
similar_names = []
for i in list_new_players :
    for j in list_liquipedia_name:
        if i == j:
            pass
        elif  Levenshtein.jaro(i,j) > 0.75:
            similar_names.append([j,i])
df_similar_names = pd.DataFrame(similar_names,columns=['liquipedia_name','import_name'])

#### Controll
***This list must be controlled before pushed!***

In [13]:
df_similar_names

,liquipedia_name,import_name


Do some magic here

In [14]:
df_similar_names = df_similar_names.iloc[[0,1]]

df_similar_names

IndexError: positional indexers are out-of-bounds

#### Data push
**Hier ist der Push in die Database, be sure**

The datafram has to be established

In [15]:
## ACHTUNG é ist das selbe wie e
df_similar_names_push = pd.merge(
                                df_similar_names,df_liquipedia_names[['liquipedia_name','player_id']], how='left',
                                left_on= 'liquipedia_name', right_on='liquipedia_name').drop(columns=['liquipedia_name'])


df_player_import = player_import.copy()
df_player_import['name_lower'] = df_player_import['import_name'].apply(lambda x : x.lower())
df_player_import = df_player_import.drop_duplicates(subset=['name_lower'])
df_similar_names_push = (pd.merge(df_similar_names_push,df_player_import,how='left', left_on='import_name',right_on='name_lower')
                         .drop(columns=['import_name_x','name_lower'])
                         .rename(columns={'import_name_y': 'player_name'}))

In [16]:
sql_int.push_to_database(df_similar_names_push,'alternative_names',sql_int.get_engine_alchemy(),'tft_tables')


The alternative_names table was imported successfully.


## Push new names in *players*

In [17]:
df_new_players = pd.merge(df_new_players,df_similar_names, how='left',
                              left_on= 'import_name', right_on='import_name')  ## on player name cause all with same liquipedia_name are already out
df_new_players = df_new_players[df_new_players['liquipedia_name'].isna()]
df_new_players = df_new_players.drop(columns=['liquipedia_name'])
new_players_push = (pd.merge(df_new_players['import_name'], df_player_import, how="left", left_on='import_name',right_on='name_lower')
                    .drop(columns=['import_name_x','name_lower'])
                    .rename(columns={'import_name_y' : 'liquipedia_name','country_flag' : 'country_name'}))


In [18]:

sql_int.push_to_database(new_players_push,'players',sql_int.get_engine_alchemy(),'tft_tables')

The players table was imported successfully.


## Push these new names in alternative_names

In [19]:
sql_query = f'''select liquipedia_name, player_id from {schema}.players;'''
df_liquipedia_names = pd.DataFrame(sql_int.get_data_mysql(sql_query),
                                columns=['liquipedia_name','player_id'])

In [20]:
df_alternative_names_push = (pd.merge(new_players_push['liquipedia_name'],df_liquipedia_names,how='left', left_on='liquipedia_name',right_on='liquipedia_name')
                             .rename(columns={'liquipedia_name':'player_name'})
                             )

In [21]:

sql_int.push_to_database(df_alternative_names_push,'alternative_names',sql_int.get_engine_alchemy(),'tft_tables')

The alternative_names table was imported successfully.


In [ ]:
df #updating data in dataframe wäre der dataframe mit den liquid_namen mit flagge. (die vorher keine gehabt haben)

s_update = "" #String of updations

# Loop through the data frame

for i in range(len(df)):
    s_update += "update players set country_name = '%s' where liquipedia_name = '%s';" %(df[country_name][i], df[liquipedia_name][i])